In [3]:
import numpy as np
import pandas as pd

In [9]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [11]:
df = pd.read_csv(r"D:\Machine Learning 100 days\Day_28_column_transformation\covid_toy.csv")

In [13]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [16]:
df['cough'].value_counts()

cough
Mild      62
Strong    38
Name: count, dtype: int64

In [18]:
df['city'].value_counts()

city
Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: count, dtype: int64

In [22]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [26]:
#age and fever-numerical
#gender , cough and city - categorial(gender and city-nominal(onehotencoding) and cough-ordinal(ordinal encoding))
#in fever values are missing
#had_covid-label encoding we will not do here

In [30]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(df.drop(columns=['has_covid']),df['has_covid'],test_size=0.2)

In [32]:
X_train

,age,gender,fever,cough,city
76,80,Male,100.0,Mild,Bangalore
47,18,Female,104.0,Mild,Bangalore
63,10,Male,100.0,Mild,Bangalore
7,20,Female,NaN,Strong,Mumbai
48,66,Male,99.0,Strong,Bangalore
...,...,...,...,...,...
20,12,Male,98.0,Strong,Bangalore
82,24,Male,98.0,Mild,Kolkata
55,81,Female,101.0,Mild,Mumbai
57,49,Female,99.0,Strong,Bangalore


In [34]:
Y_train

76    Yes
47     No
63     No
7     Yes
48     No
     ... 
20     No
82    Yes
55    Yes
57     No
91    Yes
Name: has_covid, Length: 80, dtype: object

# 1.Aam Zindagi(without using columns transformation)

In [43]:
#adding simple imputer to fever col
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])

#also train the test data
X_test_fever = si.fit_transform(X_test[['fever']])

X_train_fever  #missing values has been replaced by the mean of data

array([[100.        ],
       [104.        ],
       [100.        ],
       [100.81690141],
       [ 99.        ],
       [101.        ],
       [ 99.        ],
       [101.        ],
       [ 98.        ],
       [100.81690141],
       [104.        ],
       [ 99.        ],
       [ 98.        ],
       [100.        ],
       [104.        ],
       [104.        ],
       [ 98.        ],
       [ 98.        ],
       [104.        ],
       [ 99.        ],
       [102.        ],
       [100.81690141],
       [102.        ],
       [102.        ],
       [100.        ],
       [101.        ],
       [100.81690141],
       [ 98.        ],
       [101.        ],
       [103.        ],
       [101.        ],
       [103.        ],
       [100.        ],
       [102.        ],
       [104.        ],
       [100.81690141],
       [103.        ],
       [101.        ],
       [ 98.        ],
       [100.        ],
       [ 99.        ],
       [ 98.        ],
       [102.        ],
       [ 98

In [45]:
X_train_fever.shape

(80, 1)

In [59]:
#ordinalencoding->cough
oe = OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])

#also the test data
X_test_cough = oe.fit_transform(X_test[['cough']])

X_train_cough.shape

(80, 1)

In [67]:
#onehotencoding->gender, city
ohe = OneHotEncoder(drop='first',sparse_output=False) #drop='fitst' removing 1st column
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])

#also the test data
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])

X_train_gender_city.shape

(80, 4)

In [69]:
#now we have concatenate all the columns

In [71]:
# Extracting Age
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

X_train_age.shape

(80, 1)

In [73]:
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)
# also the test data
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)

X_train_transformed.shape

(80, 7)

# Mentos Zindagi(using column transformer

In [78]:
from sklearn.compose import ColumnTransformer

In [84]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(sparse_output=False,drop='first'),['gender','city'])
],remainder='passthrough')

In [86]:
transformer.fit_transform(X_train)

array([[100.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  80.        ],
       [104.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  18.        ],
       [100.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  10.        ],
       [100.81690141,   1.        ,   0.        ,   0.        ,
          0.        ,   1.        ,  20.        ],
       [ 99.        ,   1.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  66.        ],
       [101.        ,   0.        ,   1.        ,   1.        ,
          0.        ,   0.        ,  15.        ],
       [ 99.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  72.        ],
       [101.        ,   1.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  34.        ],
       [ 98.        ,   1.        ,   0.        ,   1.        ,
          0.    

In [88]:
transformer.fit_transform(X_train).shape

(80, 7)

In [90]:
transformer.fit_transform(X_test)

array([[101.        ,   0.        ,   1.        ,   1.        ,
          0.        ,   0.        ,  19.        ],
       [100.        ,   0.        ,   1.        ,   1.        ,
          0.        ,   0.        ,  27.        ],
       [101.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   1.        ,  65.        ],
       [101.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,   8.        ],
       [102.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  64.        ],
       [100.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  11.        ],
       [100.        ,   0.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  55.        ],
       [ 98.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   1.        ,  65.        ],
       [ 99.        ,   1.        ,   0.        ,   0.        ,
          1.    

In [92]:
transformer.fit_transform(X_test).shape

(20, 7)